In [2]:
import csv
import pandas as pd
from pyinflect import getAllInflections, getInflection
from copy import deepcopy

In [ ]:
#pd.show_versions()
#print(pd.__version__)

In [3]:
#set the mode to view the df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
### THIS IS AN ORIGINAL NGRAM WITH POSSIBLE DUPLICATES. If you read this file, clean the data with extra steps: 
ngram = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_v3.xlsx', index_col=None)
ngram = ngram.set_index('entry')
entrylist = ngram.index.tolist() #list of all entries in Google Ngram spreadsheet
ngram.shape

(120508, 5)

In [8]:
ngram.tail()

transitive  intransitive  \
entry                             
•            9486         15673   
€               0            30   
™             569             0   
■              11            37   
□               0            21   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
### find the duplicate member of entry
#duplicates = [i for i in entrylist if entrylist.count(i)!=1] NEVER DO THIS WAY; VERY INEFFICIENT
ngram_clean = ngram[~ngram.index.duplicated()]
#del ngram

### save the new ngram to a file (Next time: just start from this file)
ngram_clean.to_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_v3_clean.csv')


In [6]:
ngram_clean.shape

(120506, 5)

In [ ]:
### If Ngram spreadsheet has already existed, start from here
#ngram_clean = pd.read_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Transitivity_from_GoogleNgram_NoDuplicate.csv', index_col=None)

In [7]:
ngram_dict = ngram_clean.to_dict(orient='index') ##ngram as dictionary

In [8]:
ngram_dict['fix']['transitive'] > ngram_dict['fix']['intransitive']

True

In [9]:
ngram_dict['use']['transitive'] > ngram_dict['use']['intransitive']

True

In [11]:
ngram_dict['use']['transitive']/ngram_dict['use']['totalcount']

0.8347578807017887

In [25]:
##load verbvector spreadsheet
#vv = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.13.xlsx', index_col='Lemma')
vv = pd.read_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.18_wb.xlsx', index_col='Lemma')

In [32]:
#vv.head()
vv = vv.drop(columns=['In'])
#len(vv.index.tolist())

In [33]:
vv.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
Lemma,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,unergative,78.0,4.40,16.0,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,88.78,11.17,90.01,9.99,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,unergative,16.0,43.00,2.0,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,95.67,4.32,94.45,5.55,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,unergative,18.0,34.00,44.0,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,76.27,23.65,71.00,29.00,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,unergative,95.0,0.78,4.7,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,94.18,5.81,92.30,7.70,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44
leave,6,6,11.245,2.53,5.58,CREA,both,NaN,"unergative PP arg, transitive",unergative,unergative,13.0,8.80,40.0,354.0,tr,10.0,49.0,200.0,tr,polysemous,tr,unergativefailed,38.02,61.13,57.57,42.43,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of change of possession, verbs of change of possession, verbs of change of possession, hold and keep verbs, verbs of motion, verbs of motion","13.3FUT:have, 13.4.1entrust, 13.5.1get, 15.2keep, 51.1advance, 51.2abandon",0.97,5,4,33,1.70,1.00,3,1,565.818,0.97,617.846,1.000,1630.15,502.36


In [27]:
#[i for i in vv.index.tolist() if vv.index.tolist().count(i)>1]

[]

In [34]:
vv_dict = vv.to_dict(orient='index')
verblist = list(vv_dict.keys()) ##get the list of all verbs

In [36]:
vv_dict

{'work': {'GrandIndex': 1,
  'Index': 1,
  'LogFreqHAL': 13.02,
  'Concreteness': 3.48,
  'AgeofAcqsn': 5.86,
  'Database': 'CREA',
  'Trans_Intrans': 'both',
  'SynClassLitrt': 'unergative',
  'SynClassMemo': 'unergative',
  'SynClassSK': 'unergative',
  'SynClassNew': 'unergative',
  'unerg_pb': 78.0,
  'unacc_pb': 4.4,
  'tr_pb': 16.0,
  'totalN_pb': 431.0,
  'most_frequent_pb': 'unerg',
  'intr_g': nan,
  'tr_g': nan,
  'totalN_g': nan,
  'most_frequent_g': nan,
  'Note': nan,
  'SynClassCorpus': 'unerg',
  'SynClassFinal': 'unergative',
  'intr_google': 88.78,
  'trans_google': 11.17,
  'google1950_intrans': 90.01,
  'google1950_trans': 9.99,
  'AspClass_Consensus': 'Process',
  'SemanticType_JB': 'other - general process',
  'SemanticType_JB_edit': 'other - general process',
  'Typicality_JB': 5.0,
  'SemanticType_LC': 'other',
  'Typicality_LC': nan,
  'SemClass_Consensus': 'other',
  'CONSENSUS_JB': 'other',
  'SemanticTypeLevin': 'verbs of creating and transformation',
  'Sema

In [15]:
# ## convert the unerga_pb, unacc_pb, tr_pb, intr_g, tr_g as percentage
# for key, value in vv_dict.items():
#     for k, v in value.items():
#         if k in ['unerg_pb', 'unacc_pb', 'tr_pb', 'intr_g', 'tr_g', 'intr_google', 'trans_google'] and v != 'notfound':
#             #v_new = float(v*100)
#             v_new = round(float(v*100), 2)
#             vv_dict[key][k] = v_new

In [38]:
### construct an inflection dictionary, i.e., {run: ['runs, ran, running']}
inflection_dict = {}
for verb in verblist:
    inflection_dict[verb] = []
    inflections = getAllInflections(verb, pos_type='V')
    for k, v in inflections.items():
        v_list = list(v) ##tuple to list
        if not v_list[0].endswith('ing'): ###not going to keep a progressive form
            inflection_dict[verb].append(v_list[0]) ### if there are >1 variant for a given inflection, only keep 1st
    liszt = inflection_dict[verb] ##to drop duplicates, convert it into a set
    inflection_dict[verb] = list(set(liszt))

inflection_dict['arrive']

['arrive', 'arrived', 'arrives']

In [39]:
#getAllInflections('fix', pos_type='V')

In [45]:
## update the vv dictionary (our verb vector)
for k, v in vv_dict.items(): ### i.e., k='work', v={'GrandIndex':1, ...}
    trans_count = 0
    intrans_count = 0
    total_count = 0
    wordforms = inflection_dict[k] ##wordforms = ['work', 'works', 'worked', ...]
    for wordform in wordforms: 
        try: 
            trans_count += ngram_dict[wordform]['transitive']
            intrans_count += ngram_dict[wordform]['intransitive']
            total_count += ngram_dict[wordform]['totalcount']
#             trans_percent = round((trans_count/total_count)*100, 2)
#             intrans_percent = round((intrans_count/total_count)*100, 2)    
        except: 
            print (wordform, wordform in ngram_dict.keys())
        
    trans_percent = round((trans_count/total_count)*100, 2)
    intrans_percent = round((intrans_count/total_count)*100, 2)    

    v['google1950_45gram_trans'] = trans_percent
    v['google1950_45gram_intrans'] = intrans_percent

burps False
oinks False
oinked False


In [47]:
##save vv_dict to a df
df_out = pd.DataFrame.from_dict(vv_dict, orient='index')

In [48]:
def rearrange_col(dataframe, col_to_move, reference_col, right=True):
    col_list = dataframe.columns.values.tolist()
    col_list2 = [x for x in col_list if x != col_to_move]
    reference_idx = col_list2.index(reference_col)
    if right==True:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.append(col_to_move)
        return dataframe[col_list3]
    else:
        col_list3 = []
        for y in col_list2:
            col_list3.append(y)
            if y == reference_col:
                col_list3.insert(-2, col_to_move)
        return  dataframe[col_list3]

In [49]:
df_out.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F,google1950_45gram_trans,google1950_45gram_intrans
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,unergative,78.0,4.40,16.0,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,88.78,11.17,90.01,9.99,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67,12.13,87.87
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,unergative,16.0,43.00,2.0,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,95.67,4.32,94.45,5.55,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN,6.13,93.87
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,unergative,18.0,34.00,44.0,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,76.27,23.65,71.00,29.00,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70,33.50,66.50
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,unergative,95.0,0.78,4.7,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,94.18,5.81,92.30,7.70,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44,8.51,91.49
leave,6,6,11.245,2.53,5.58,CREA,both,NaN,"unergative PP arg, transitive",unergative,unergative,13.0,8.80,40.0,354.0,tr,10.0,49.0,200.0,tr,polysemous,tr,unergativefailed,38.02,61.13,57.57,42.43,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of change of possession, verbs of change of possession, verbs of change of possession, hold and keep verbs, verbs of motion, verbs of motion","13.3FUT:have, 13.4.1entrust, 13.5.1get, 15.2keep, 51.1advance, 51.2abandon",0.97,5,4,33,1.70,1.00,3,1,565.818,0.97,617.846,1.000,1630.15,502.36,62.22,37.78


In [50]:
df_out2 = rearrange_col(df_out, 'google1950_45gram_intrans', 'google1950_trans')
df_out3 = rearrange_col(df_out2, 'google1950_45gram_trans', 'google1950_45gram_intrans')
df_out3.head()

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,google1950_45gram_intrans,google1950_45gram_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
work,1,1,13.020,3.48,5.86,CREA,both,unergative,unergative,unergative,unergative,78.0,4.40,16.0,431.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,88.78,11.17,90.01,9.99,87.87,12.13,Process,other - general process,other - general process,5.0,other,NaN,other,other,verbs of creating and transformation,26.5knead,0.57,4,9,25,1.50,1.20,3,1,633.152,1.00,521.889,1.000,2421.10,1034.67
go,2,2,12.923,3.15,3.37,CREA,intransitive only,unergative,unergative PP arg,unergative,unergative,16.0,43.00,2.0,1029.0,unacc,NaN,NaN,NaN,NaN,NaN,unacc,unergative,95.67,4.32,94.45,5.55,93.87,6.13,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of existence, verbs of motion","47.7meander, 51.1advance",0.98,2,17,48,1.35,1.00,2,1,597.688,0.97,524.071,1.000,974.49,NaN
run,3,3,12.316,4.31,4.47,CREA,both,unergative,unergative,unergative,unergative,18.0,34.00,44.0,374.0,tr,NaN,NaN,NaN,NaN,NaN,tr,unergative,76.27,23.65,71.00,29.00,66.50,33.50,Process,body mvt or posture,body movement or posture,5.0,manner of motion,5.0,manner of motion,manner of motion,"verbs of creating and transformation, verbs of existence, verbs of existence, verbs of motion","26.3prepare, 47.5.1swarm, 47.7meander, 51.3.2run",0.89,3,19,40,1.00,1.00,3,1,596.647,1.00,559.955,1.000,386.53,229.70
live,5,5,11.763,3.57,6.10,CREA,intransitive only,unergative/unaccusative,unergative PP arg,unergative,unergative,95.0,0.78,4.7,129.0,unerg,NaN,NaN,NaN,NaN,NaN,unerg,unergative,94.18,5.81,92.30,7.70,91.49,8.51,State,existence,existence,5.0,existence,5.0,existence,existence,"verbs of ingesting, lodge verbs, verbs of existence","39.6gorge, 46lodge, 47.1exist",0.89,4,16,21,1.15,1.25,3,1,629.886,1.00,553.769,0.963,3645.64,606.44
leave,6,6,11.245,2.53,5.58,CREA,both,NaN,"unergative PP arg, transitive",unergative,unergative,13.0,8.80,40.0,354.0,tr,10.0,49.0,200.0,tr,polysemous,tr,unergativefailed,38.02,61.13,57.57,42.43,37.78,62.22,Achievement,ch of location,ch of location,5.0,ch of location,5.0,ch of location,ch of location,"verbs of change of possession, verbs of change of possession, verbs of change of possession, hold and keep verbs, verbs of motion, verbs of motion","13.3FUT:have, 13.4.1entrust, 13.5.1get, 15.2keep, 51.1advance, 51.2abandon",0.97,5,4,33,1.70,1.00,3,1,565.818,0.97,617.846,1.000,1630.15,502.36


In [52]:
## save the df to a csv (###v2.14 = old version where ngram containing 'nsubjpass' was NOT identified as transitive)
#df_out3.to_csv('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.15.csv')
df_out3.to_excel('/Users/songheekim/Google Drive/Primary/Projects/VerbVector/Verblists/verblist_v2.19.xlsx')

In [53]:
df_out3[df_out3.index=='find']

,GrandIndex,Index,LogFreqHAL,Concreteness,AgeofAcqsn,Database,Trans_Intrans,SynClassLitrt,SynClassMemo,SynClassSK,SynClassNew,unerg_pb,unacc_pb,tr_pb,totalN_pb,most_frequent_pb,intr_g,tr_g,totalN_g,most_frequent_g,Note,SynClassCorpus,SynClassFinal,intr_google,trans_google,google1950_intrans,google1950_trans,google1950_45gram_intrans,google1950_45gram_trans,AspClass_Consensus,SemanticType_JB,SemanticType_JB_edit,Typicality_JB,SemanticType_LC,Typicality_LC,SemClass_Consensus,CONSENSUS_JB,SemanticTypeLevin,SemanticTypeEntry,Percentage_dom_PoS,Length,Ortho_N,Phono_N,OLD,PLD,NPhon,NSyll,I_Mean_RT,I_Mean_Accuracy,I_NMG_Mean_RT,I_NMG_Mean_Accuracy,N2_F,N3_F
find,270,4,12.845,2.63,5.78,CREA,transitive only,transitive,transitive,transitive,?transitive,2.0,8.5,78.0,457.0,tr,1.5,87.0,200.0,tr,NaN,tr,transitive,39.38,60.15,51.56,48.44,45.1,54.9,Achievement,cognitive event,cognitive event,3.0,cognitive event,4.0,cognitive event,cognitive event,"verbs of change of possession, verbs with predicative complements","13.5.1get, 29.4declare",1.0,4,15,26,1.15,1.15,4,1,619.562,0.94,580.333,1.0,2150.47,997.75


In [ ]:
### for testing
# trans_count = 0
# intrans_count = 0
# total_count = 0 
# for wordform in inflection_dict['work']:
#     print (wordform, ngram_dict[wordform])
#     trans_count += ngram_dict[wordform]['transitive']
#     intrans_count += ngram_dict[wordform]['intransitive']
#     total_count += ngram_dict[wordform]['totalcount']
#     trans_percent = round((trans_count/total_count)*100, 2)
#     intrans_percent = round((intrans_count/total_count)*100,2)

# print(trans_count)
# print(intrans_count)
# print(total_count)
# print(trans_percent)
# print(intrans_percent)